# Split Images into Tiles Technique

# 1. Grid-based Tiling with Excess Pixel Handling

In [1]:
import os
import cv2
import tifffile
import numpy as np

size = 256

def split_image_into_tiles(image_path, mask_path, tile_size):
    img = tifffile.imread(image_path)
    mask = tifffile.imread(mask_path)

    mask = mask[:, :, 0] if len(mask.shape) == 3 else mask
    print(f"Original Image Dimensions: {img.shape}")
    tiles_img = []
    tiles_mask = []

    for x in range(0, img.shape[1], tile_size):
        for y in range(0, img.shape[0], tile_size):
            tile_img = img[y:y+tile_size, x:x+tile_size, :] if len(img.shape) == 3 else img[y:y+tile_size, x:x+tile_size]
            tile_mask = mask[y:y+tile_size, x:x+tile_size]

            # Resize to the desired size
            tile_img = cv2.resize(tile_img, (size, size))
            tile_mask = cv2.resize(tile_mask, (size, size))
            tile_mask = (tile_mask > 0).astype(np.uint8)

            tiles_img.append(tile_img)
            tiles_mask.append(tile_mask)

    # Calculate the number of tiles needed to form a perfect square
    num_tiles = len(tiles_img)
    perfect_square_size = int(np.ceil(np.sqrt(num_tiles)))
    total_tiles_needed = perfect_square_size**2
    num_tiles_to_add = total_tiles_needed - num_tiles

    # Create false tiles with placeholder values
    false_tile_img = np.zeros((size, size, img.shape[2]), dtype=np.uint8)
    false_tile_mask = np.zeros((size, size), dtype=np.uint8)

    for _ in range(num_tiles_to_add):
        tiles_img.append(false_tile_img)
        tiles_mask.append(false_tile_mask)

    tiles_img = np.array(tiles_img)
    tiles_mask = np.array(tiles_mask)

    return tiles_img, tiles_mask

# Function to load data
def load_data(image_dir, mask_dir, tile_size):
    images = []
    masks = []

    # Sort filenames to ensure consistency
    image_filenames = sorted(os.listdir(image_dir))
    mask_filenames = sorted(os.listdir(mask_dir))

    for image_filename in image_filenames:
        if image_filename.endswith(".TIF"):
            mask_filename = image_filename.replace(".TIF", "_mask.TIF")

            if mask_filename in mask_filenames:
                image_path = os.path.join(image_dir, image_filename)
                mask_path = os.path.join(mask_dir, mask_filename)

                print(f"Processing Image: {image_filename}, Mask: {mask_filename}")

                img, mask = split_image_into_tiles(image_path, mask_path, tile_size)

                images.extend(img)
                masks.extend(mask)

    return np.array(images), np.array(masks)

# Define paths
image_dir = "datasets/images"
mask_dir = "datasets/masks"

# Tile size 
tile_size = size

# Load data
tiles_img, tiles_mask = load_data(image_dir, mask_dir, tile_size)

# Display the number of tiles produced
print(f"Number of tile images: {tiles_img.shape[0]}")
print(f"Number of tile masks: {tiles_mask.shape[0]}")


Processing Image: 2.TIF, Mask: 2_mask.TIF
Original Image Dimensions: (7151, 7941, 3)
Number of tile images: 900
Number of tile masks: 900


# 2. Reshape As Rectangle

In [2]:
import os
import cv2
import tifffile
import numpy as np


size = 256
num_channels = 3  # RGB Band


def split_image_into_tiles(image_path, mask_path, tile_size):
    img = tifffile.imread(image_path)
    mask = tifffile.imread(mask_path)

    mask = mask[:, :, 0] if len(mask.shape) == 3 else mask
    print(f"Original Image Dimensions: {img.shape}")
    tiles_img = []
    tiles_mask = []

    for x in range(0, img.shape[1], tile_size):
        for y in range(0, img.shape[0], tile_size):
            tile_img = img[y:y+tile_size, x:x+tile_size, :] if len(img.shape) == 3 else img[y:y+tile_size, x:x+tile_size]
            tile_mask = mask[y:y+tile_size, x:x+tile_size]

            tile_img = cv2.resize(tile_img, (size, size))
            tiles_img.append(tile_img)

            tile_mask = cv2.resize(tile_mask, (size, size))
            tile_mask = (tile_mask > 0).astype(np.uint8)
            tiles_mask.append(tile_mask)

    # Ensure the number of tiles is a perfect square
    num_tiles = len(tiles_img)
    perfect_square_size = int(np.ceil(np.sqrt(num_tiles)))

    tiles_img = np.array(tiles_img).reshape(-1, size, size, num_channels)
    tiles_mask = np.array(tiles_mask).reshape(-1, size, size)

    return tiles_img, tiles_mask

# Function to load data
def load_data(image_dir, mask_dir, tile_size):
    images = []
    masks = []

    # Sort filenames to ensure consistency
    image_filenames = sorted(os.listdir(image_dir))
    mask_filenames = sorted(os.listdir(mask_dir))

    for image_filename in image_filenames:
        if image_filename.endswith(".TIF"):
            mask_filename = image_filename.replace(".TIF", "_mask.TIF")

            if mask_filename in mask_filenames:
                image_path = os.path.join(image_dir, image_filename)
                mask_path = os.path.join(mask_dir, mask_filename)

                print(f"Processing Image: {image_filename}, Mask: {mask_filename}")

                img, mask = split_image_into_tiles(image_path, mask_path, tile_size)

                images.extend(img)
                masks.extend(mask)

    return np.array(images), np.array(masks)

# Define  paths
image_dir = "datasets/images"
mask_dir = "datasets/masks"

# Tile size 
tile_size = size

# Load data
tiles_img, tiles_mask = load_data(image_dir, mask_dir, tile_size)

# Display the number of tiles produced
print(f"Number of tile images: {tiles_img.shape[0]}")
print(f"Number of tile masks: {tiles_mask.shape[0]}")


Processing Image: 2.TIF, Mask: 2_mask.TIF
Original Image Dimensions: (7151, 7941, 3)
Number of tile images: 896
Number of tile masks: 896


# 3. Padding Technique

In [3]:
import os
import cv2
import tifffile
import numpy as np

size = 256

def split_image_into_tiles(image_path, mask_path, tile_size):
    img = tifffile.imread(image_path)
    mask = tifffile.imread(mask_path)

    mask = mask[:, :, 0] if len(mask.shape) == 3 else mask
    print(f"Original Image Dimensions: {img.shape}")
    tiles_img = []
    tiles_mask = []

    for x in range(0, img.shape[1], tile_size):
        for y in range(0, img.shape[0], tile_size):
            tile_img = img[y:y+tile_size, x:x+tile_size, :] if len(img.shape) == 3 else img[y:y+tile_size, x:x+tile_size]
            tile_mask = mask[y:y+tile_size, x:x+tile_size]

            # Resize to the desired size
            tile_img = cv2.resize(tile_img, (size, size))
            tile_mask = cv2.resize(tile_mask, (size, size))
            tile_mask = (tile_mask > 0).astype(np.uint8)

            tiles_img.append(tile_img)
            tiles_mask.append(tile_mask)

    # Calculate the number of tiles needed to form a perfect square
    num_tiles = len(tiles_img)
    perfect_square_size = int(np.ceil(np.sqrt(num_tiles)))
    total_tiles_needed = perfect_square_size**2
    num_tiles_to_add = total_tiles_needed - num_tiles

    # Pad the list of tiles with zeros
    for _ in range(num_tiles_to_add):
        tiles_img.append(np.zeros((size, size, img.shape[2]), dtype=np.uint8))
        tiles_mask.append(np.zeros((size, size), dtype=np.uint8))

    tiles_img = np.array(tiles_img)
    tiles_mask = np.array(tiles_mask)

    return tiles_img, tiles_mask

# Function to load data
def load_data(image_dir, mask_dir, tile_size):
    images = []
    masks = []

    # Sort filenames to ensure consistency
    image_filenames = sorted(os.listdir(image_dir))
    mask_filenames = sorted(os.listdir(mask_dir))

    for image_filename in image_filenames:
        if image_filename.endswith(".TIF"):
            mask_filename = image_filename.replace(".TIF", "_mask.TIF")

            if mask_filename in mask_filenames:
                image_path = os.path.join(image_dir, image_filename)
                mask_path = os.path.join(mask_dir, mask_filename)

                print(f"Processing Image: {image_filename}, Mask: {mask_filename}")

                img, mask = split_image_into_tiles(image_path, mask_path, tile_size)

                images.extend(img)
                masks.extend(mask)

    return np.array(images), np.array(masks)

# Define  paths
image_dir = "datasets/images"
mask_dir = "datasets/masks"

# Tile size 
tile_size = size

# Load data
tiles_img, tiles_mask = load_data(image_dir, mask_dir, tile_size)

# Display the number of tiles produced
print(f"Number of tile images: {tiles_img.shape[0]}")
print(f"Number of tile masks: {tiles_mask.shape[0]}")


Processing Image: 2.TIF, Mask: 2_mask.TIF
Original Image Dimensions: (7151, 7941, 3)
Number of tile images: 900
Number of tile masks: 900
